# 데이터프레임의 데이터 조작

Pandas는 데이터 처리 및 변환을 위한 다양한 함수와 메서드를 제공한다. 여기에서는 그 중 몇가지를 예로 보인다. 전체 기능은 다음 웹사이트를 참조한다

* https://pandas.pydata.org/pandas-docs/stable/api.html

## 갯수 세기

가장 간단한 데이터 분석은 데이터의 갯수를 세는 것이다. `count` 메서드를 사용한다. NaN 값은 세지 않는다.

In [1]:
import numpy as np
import pandas as pd

In [2]:
s = pd.Series(range(10))
s[3] = np.nan
s

0    0.0
1    1.0
2    2.0
3    NaN
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
dtype: float64

In [3]:
s.count() # nan값 카운팅 X

9

In [5]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(5, size=(4, 4)))
df.iloc[2,3] = np.nan
df

0  1  2    3
0  3  4  0  1.0
1  3  0  0  1.0
2  4  4  1  NaN
3  4  2  4  3.0

In [5]:
df.count()

0    4
1    4
2    4
3    3
dtype: int64

## 카테고리 값 세기

**시리즈**의 값이 정수, 문자열, 카테고리 값인 경우에는 `value_counts` 메서드로 각각의 값이 나온 횟수를 셀 수 있다.

In [6]:
np.random.seed(1)
s = pd.Series(np.random.randint(6, size=100))
s.tail()

95    4
96    5
97    2
98    4
99    3
dtype: int32

In [8]:
s.value_counts() # 각 정수 별 개수 

1    22
0    18
4    17
5    16
3    14
2    13
dtype: int64

In [12]:
s.sort_index?

In [13]:
s.value_counts().sort_index()

0    18
1    22
2    13
3    14
4    17
5    16
dtype: int64

위 코드에서 `sort_index`는 인덱스 라벨을 기준으로 정렬하는 명령이다.

## 행/열 합계

In [14]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(10, size=(4, 8)))
df

0  1  2  3  4  5  6  7
0  5  8  9  5  0  0  1  7
1  6  9  2  4  5  2  4  2
2  4  7  7  9  1  7  0  6
3  9  9  7  6  9  1  0  1

### 행 합계

In [25]:
df.sum(axis=1) # axis =1 행 / axis = 0 열

0    35
1    34
2    41
3    42
dtype: int64

In [26]:
df["Sum"] = df.sum(axis=1)
df

0  1  2  3  4  5  6  7  Sum
0  5  8  9  5  0  0  1  7   35
1  6  9  2  4  5  2  4  2   34
2  4  7  7  9  1  7  0  6   41
3  9  9  7  6  9  1  0  1   42

### 열 합계

In [31]:
df.sum(axis=0) #df.sum(axis=0)

0       24
1       33
2       25
3       24
4       15
5       10
6        5
7       16
Sum    152
dtype: int64

In [32]:
df.loc["Total", :] = df.sum()
df

0     1     2     3     4     5    6     7    Sum
0       5.0   8.0   9.0   5.0   0.0   0.0  1.0   7.0   35.0
1       6.0   9.0   2.0   4.0   5.0   2.0  4.0   2.0   34.0
2       4.0   7.0   7.0   9.0   1.0   7.0  0.0   6.0   41.0
3       9.0   9.0   7.0   6.0   9.0   1.0  0.0   1.0   42.0
Total  24.0  33.0  25.0  24.0  15.0  10.0  5.0  16.0  152.0

## `apply` 변환

행이나 열 단위로 더 복잡한 처리를 하고 싶을 때는 `apply` 메서드를 사용한다. 인수로 행 또는 열을 받는 함수를 `apply` 메서드의 인수로 넣으면 각 열(또는 행)을 반복하여 그 함수에 적용시킨다. 

데이터프레임.apply(함수, axis= )

In [45]:
df = pd.DataFrame({
        'A': [1, 3, 4, 3, 4],
        'B': [2, 3, 1, 2, 3],
        'C': [1, 5, 2, 4, 4]
    })
df

A  B  C
0  1  2  1
1  3  3  5
2  4  1  2
3  3  2  4
4  4  3  4

예를 들어 각 열의 최대값과 최소값의 차이를 구하고 싶으면 다음과 같은 람다 함수를 넣는다.

In [40]:
df.apply(lambda x: x.max() - x.min())

A    3
B    2
C    4
dtype: int64

만약 행에 대해 적용하고 싶으면 `axis` = 1을 사용한다.

In [41]:
df.apply(lambda x: x.max() - x.min(), axis=1)

0    1
1    2
2    3
3    2
4    1
dtype: int64

각 열에 대해 어떤 값이 얼마나 사용되었는지 알고 싶다면 `value_counts` 함수를 넣을 수도 있다.

In [46]:
df.apply(pd.value_counts)

A    B    C
1  1.0  1.0  1.0
2  NaN  2.0  1.0
3  2.0  2.0  NaN
4  2.0  NaN  2.0
5  NaN  NaN  1.0

NaN 값은 `fillna` 함수를 사용하여 원하는 값으로 바꿀 수 있다.

In [47]:
df.apply(pd.value_counts).fillna(0)

A    B    C
1  1.0  1.0  1.0
2  0.0  2.0  1.0
3  2.0  2.0  0.0
4  2.0  0.0  2.0
5  0.0  0.0  1.0

## 실수 값을 카테고리 값으로 변환

실수 값을 크기 기준으로 하여 카테고리 값으로 변환하고 싶을 때는 다음과 같은 명령을 사용한다.

* `cut`: 실수 값의 경계선을 지정하는 경우
* `qcut`: 갯수가 똑같은 구간으로 나누는 경우

예를 들어 다음과 같은 나이 데이터가 있다고 하자.

In [48]:
ages = [0, 2, 10, 21, 23, 37, 31, 61, 20, 41, 32]

이 값을 다음과 같이 카테고리 값으로 바꿀 수 있다. bins 영역을 넘는 값은 NaN으로 처리된다.

In [49]:
bins = [1, 15, 25, 35, 60, 190]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
cats = pd.cut(ages, bins, labels=labels)
cats

[NaN, 미성년자, 미성년자, 청년, 청년, ..., 중년, 노년, 청년, 장년, 중년]
Length: 11
Categories (5, object): [미성년자 < 청년 < 중년 < 장년 < 노년]

결과로 반환되는 값은 `Categorical` 클래스 객체이다. 이 객체는 `categories` 속성에 라벨 문자열을  `codes` 속성에 정수로 인코딩한 카테고리 값을 가진다.

In [50]:
type(cats)

pandas.core.categorical.Categorical

In [54]:
cats.categories

Index(['미성년자', '청년', '중년', '장년', '노년'], dtype='object')

In [55]:
cats.codes

array([-1,  0,  0,  1,  1,  3,  2,  4,  1,  3,  2], dtype=int8)

In [56]:
df = pd.DataFrame(ages, columns=["ages"])
df["age_cat"] = pd.cut(df.ages, bins, labels=labels)
df

ages age_cat
0      0     NaN
1      2    미성년자
2     10    미성년자
3     21      청년
4     23      청년
5     37      장년
6     31      중년
7     61      노년
8     20      청년
9     41      장년
10    32      중년

`qcut` 명령은 같은 데이터를 가진 구간으로 나눈다. 예를 들어 다음 코드는 1000개의 데이터를 4개의 구간으로 나누는데 각 구간은 250개씩의 데이터를 가진다.

In [58]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4, labels=["Q1", "Q2", "Q3", "Q4"])
cats

[Q3, Q1, Q1, Q1, Q2, ..., Q3, Q4, Q3, Q3, Q2]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [59]:
pd.value_counts(cats)

Q4    250
Q3    250
Q2    250
Q1    250
dtype: int64